In [16]:
# library

import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup as bs

In [2]:
def get_news_list(item_code, page_no):
    # requests 필요한 재료
    url = f'https://finance.naver.com/item/news_news.nhn?code={item_code}&page={page_no}&sm=title_entity_id.basic&clusterId='
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
    
    # 스크랩핑해와서 읽어내기
    response = requests.get(url, headers=headers)
    html = bs(response.text, 'lxml')
    temp = html.select("table")
    df_news = pd.read_html(str(temp))
    
    # nan값, 쓸수 없는 컬럼 제거
    df_news = df_news[0].dropna()
    df_news = df_news.drop(df_news[df_news["정보제공"] == df_news["날짜"]].index)
    
    return df_news
    

In [3]:
def get_item_news(item_code, item_name):
    
    # web page 시작번호
    page_no = 1
    
    # 데이터를 저장할 리스트 
    news_list = []
    
    # 리스트에 데이터 저장하기
    while True:
        temp = get_news_list(item_code, page_no)
        news_list.append(temp)
        
        page_no = page_no + 1
        
        random_sleep = np.random.uniform(0, 0.5)
        time.sleep(random_sleep)
        
        # 일정 페이지 수 이상 중지
        if page_no > 10:
            break
    
    # 종목코드, 종목명 컬럼 추가
    df = pd.concat(news_list)
    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '제목', '정보제공', '날짜']
    df = df[cols]        

    return df

In [17]:
def save_csv(df):
    # 파일명 만들기
    item_code = df.iloc[0]["종목코드"]
    item_name = df.iloc[0]["종목명"]
    date = df.iloc[0]["날짜"]
    date = date.replace(".", "-")
    date = date[:10]
    
    # 파일명 : 종목명_종목코드_최신뉴스의날짜
    filename = f"{item_name}_{item_code}_{date}.csv"
    
    # 저장
    df.to_csv(filename, index=False)
